In [1]:
import sys
import os

# Adiciona o diretório pai ao caminho de pesquisa de módulos
sys.path.append(os.path.abspath(os.path.join('..')))

In [2]:
import wandb
import torchvision.models as models

import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import lr_scheduler

import gc
import random
import pickle
from copy import deepcopy
import numpy as np

In [3]:
from torchvision import transforms

from tqdm import tqdm
from avalanche.benchmarks.classic import SplitCUB200
from utils import create_plot_images
from torch.optim.lr_scheduler import ReduceLROnPlateau

from avalanche.evaluation.metrics import ConfusionMatrix, WandBStreamConfusionMatrix
from avalanche.evaluation.metric_utils import default_cm_image_creator

In [4]:
train_transform = transforms.Compose(
    [
        transforms.Resize((128, 128)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

eval_transform = transforms.Compose(
    [
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

In [5]:
dataset_name = 'cub-200'
total_number_classes = 200
n_split_experiences = 5

# creating the benchmark (scenario object)
split_ds = SplitCUB200(
    n_experiences=n_split_experiences,
    seed=1234,
    train_transform=train_transform,
    eval_transform=eval_transform
    
)

train_stream = split_ds.train_stream
test_stream = split_ds.test_stream

train_ds = []
test_ds = []
masks = []
classes_in_experience = []

for experience in train_stream:
    print("Start of task ", experience.task_label, 'experience.current_experience', experience.current_experience)
    print('Classes in this task:', experience.classes_in_this_experience)
    masks.append([i for i in range(total_number_classes) if i not in experience.classes_in_this_experience])
    classes_in_experience.append(experience.classes_in_this_experience)
    
    # The current Pytorch training set can be easily recovered through the 
    # experience
    current_training_set = experience.dataset
    train_ds.append(current_training_set)
    
    # ...as well as the task_label
    print('Task {}'.format(experience.task_label))
    print('This task contains', len(current_training_set), 'training examples')

    # we can recover the corresponding test experience in the test stream
    current_test_set = test_stream[experience.current_experience].dataset
    print('This task contains', len(current_test_set), 'test examples')
    test_ds.append(current_test_set)
    print(80*'-')

# Getting a Sample from the first Task
batch_gen = torch.utils.data.DataLoader(train_ds[0], 
                              batch_size=9, 
                              shuffle=True, 
                              num_workers=1,
                              )
for batch in tqdm(batch_gen):
    sample = batch[0]
    target = batch[1]
    print(sample.shape)
    break

Files already downloaded and verified
Files already downloaded and verified
Start of task  0 experience.current_experience 0
Classes in this task: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
Task 0
This task contains 3000 training examples
This task contains 2864 test examples
--------------------------------------------------------------------------------
Start of task  0 experience.current_experience 1
Classes in this task: [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124]
Task 0
This task contains 749 training examples
This task contains 714 test examples
------------

  0%|                                                                                                                             | 0/334 [00:00<?, ?it/s]

torch.Size([9, 3, 128, 128])


In [6]:
from continuous_lora.models.lora_vgg19 import LoraVGG19

In [7]:
torch.cuda.is_available()

/home/everton_aleixo/Documents/effects-of-lora-on-catastrophic-forgetting/venv/lib/python3.12/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


False

In [8]:
tasks = [i for i in range(n_split_experiences)]
acc_by_task = {i: 0 for i in range(n_split_experiences)}
results_diff_models = {}
best_model = {}
cms = {}
patience = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

for loop in range(5):
    #loop=1
    if os.path.isfile('stop.txt'):
        print(f'You should continue from loop {loop}')
        break

    config = {
        "learning_rate": 0.00001,
        "weight_decay": 5.e-4,
        "batch_size": 8,
        "optimizer": "Adam",
        "architecture": "LoraVGG19",
        "dataset": f'{dataset_name.upper()}',
        "epochs": 300,
        "lr_schedule": "ReduceLROnPlateau - Patience 10 - Monitoring Val Accuracy",
        "description": "Testing CUBs200 splited into 5 tasks."
    }
    wandb.init(
        # set the wandb project where this run will be logged
        project=f"cf-lora-{dataset_name}-loravgg-parameters",
        name=f"lora-vgg19-lr-00001-5tasks-lora-{loop}",
        
        # track hyperparameters and run metadata
        config=config
    )
    
    for model_type in ['vgg19']:
        bz=config['batch_size']
            
        torch.backends.cudnn.benchmark = False
        torch.manual_seed(0)
        random.seed(0)
        np.random.seed(0)
        base_model = models.vgg19_bn(weights="IMAGENET1K_V1")
        base_model.classifier[6] = nn.Linear(4096, total_number_classes)
        model = LoraVGG19(
            model=base_model,
            masks=masks,
            adapt_last_n_conv=16,
            adapt_last_n_linear=3
        )
        
                       
        for task in tasks:
            
            epoch_losses = {'train_acc': [], 'train_loss': [], 'test_acc': [], 'test_loss': []}
            model.to(device)
            print(40*'-', f'TASK_{task}', 40*'-')
        
            criterion = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
            lr_scheduler = ReduceLROnPlateau(optimizer, mode='max', patience=4, verbose=True)
            
            early_stop = 0
            best_val = 0
            for i in range(config['epochs']):
                g = torch.Generator()
                g.manual_seed(0)
                batch_gen = torch.utils.data.DataLoader(train_ds[task], 
                                                      batch_size=config['batch_size'], 
                                                      shuffle=True, 
                                                      num_workers=1,
                                                      )
                print(f'Training task {task} in epoch {i}. Batch size: {bz}.')
                count = 0
                total_loss = .0
                total_hit = 0
                for batch in tqdm(batch_gen):
                    sample = batch[0].to(device)
                    target = batch[1]
                    target_onehot = F.one_hot(target, num_classes=total_number_classes).to(torch.float).to(device)
    
                    y_hat = model(sample)
        
                    # Compute the loss
                    loss_training = criterion(y_hat, target_onehot)            
                    
                    # Backpropagation and optimization
                    optimizer.zero_grad()
                    loss_training.backward()
                    optimizer.step()
                    
                    total_loss += loss_training
                    total_hit += sum(np.argmax(y_hat.cpu().detach().numpy(), axis=1) == target.numpy())
    
                    sample.to('cpu')
                    target_onehot.to('cpu')
                    del sample, target_onehot
                    gc.collect()
                    torch.cuda.empty_cache()
                    
                # Evaluate in test DS after each epoch
                with torch.no_grad():
                    batch_gen_test = torch.utils.data.DataLoader(test_ds[task], 
                                                      batch_size=config['batch_size'], 
                                                      shuffle=True, 
                                                      num_workers=1,
                                                      )
                    model.eval()
                    test_loss = 0.
                    acc = 0.
                    for batch_test in tqdm(batch_gen_test):
                        sample = batch_test[0].to(device)
                        target = batch_test[1]
                        target_onehot = F.one_hot(target, num_classes=total_number_classes).to(torch.float).to(device)
            
                        y_hat = model(sample)
                        cpu_inference = y_hat.argmax(axis=1)
                        
                        test_loss += criterion(y_hat, target_onehot)
                        acc += sum(cpu_inference.cpu() == target).item()
    
                        sample.to('cpu')
                        target_onehot.to('cpu')
                        del sample, target_onehot
                        gc.collect()
                        torch.cuda.empty_cache()
    
                    test_loss = (test_loss/len(batch_gen_test)).cpu()
                    acc = acc/len(test_ds[task])
                    lr_scheduler.step(acc)
                    model.train()
                
                epoch_losses['train_acc'].append(total_hit/(len(batch_gen)*bz))
                epoch_losses['train_loss'].append((total_loss/len(batch_gen)).cpu().item())
                
                epoch_losses['test_acc'].append(acc)
                epoch_losses['test_loss'].append(test_loss.item())
        
                # if acc > (best_val+0.01):
                if acc > (best_val):
                    model.to('cpu')
                    best_model[f'{model_type}-{task}'] = deepcopy(model)
                    model.to(device)
                    best_val = acc
                    early_stop = 0
        
                if early_stop > patience:
                    break
                
                early_stop += 1
    
                wandb.log({"train_acc": (total_hit/(len(batch_gen)*bz)), "train_loss": (total_loss/len(batch_gen)), 
                           "val_acc": acc, "val_loss": test_loss, "task": (task+1),
                           "Accuracy": acc, "Loss": test_loss, 
                           "patience": early_stop, "best_val_acc": best_val, "learning_rate": optimizer.param_groups[0]['lr']
                          })
                
                print(f'Trainig acc: {total_hit/(len(batch_gen)*bz):.4}   //   Training loss: {(total_loss/len(batch_gen)):.4f}   //   Test acc: {acc:.4f}   //   Test loss: {test_loss:.4f}')
                print(f'early_stop: {early_stop}  /   Best acc: {best_val}')
                del batch_gen, batch_gen_test
                gc.collect()
                torch.cuda.empty_cache()
    
            model.to('cpu')
            del model
            model = best_model[f'{model_type}-{task}']
            del best_model[f'{model_type}-{task}']
    
            gc.collect()
            torch.cuda.empty_cache()
            
            # del criterion, optimizer, lr_scheduler 
            gc.collect()
            torch.cuda.empty_cache()
            target_task = task + 1
            if target_task < len(tasks): 
                model.change_to_task(target_task)
    
        wandb.finish()

wandb: Currently logged in as: everton (cf-lora). Use `wandb login --relogin` to force relogin


/home/everton_aleixo/Documents/effects-of-lora-on-catastrophic-forgetting/venv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


---------------------------------------- TASK_0 ----------------------------------------
Training task 0 in epoch 0. Batch size: 8.


  0%|                                                                                                                                   | 0/375 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [70]:
config = {
    "learning_rate": 0.001,
    "weight_decay": 5.e-4,
    "batch_size": 128,
    "optimizer": "Adam",
    "architecture": "LoraVGG19",
    "dataset": f'{dataset_name.upper()}',
    "epochs": 10,
    "lr_schedule": "ReduceLROnPlateau - Patience 10 - Monitoring Val Accuracy",
    "description": "Testing CUBs200 splited into 5 tasks."
}

In [71]:
torch.backends.cudnn.benchmark = False
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)
base_model = models.vgg19_bn(weights="IMAGENET1K_V1")
base_model.classifier[6] = nn.Linear(4096, total_number_classes)
model = LoraVGG19(
    model=base_model,
    masks=masks,
    adapt_last_n_conv=16,
    adapt_last_n_linear=3
)
tasks = [i for i in range(n_split_experiences)]
acc_by_task = {i: 0 for i in range(n_split_experiences)}
results_diff_models = {}
best_model = {}
cms = {}
patience = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [54]:
device, model

('cuda',
 LoraVGG19(
   (features): Sequential(
     (0): ContinuousConvLoRALayer(
       (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     )
     (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (2): ReLU(inplace=True)
     (3): ContinuousConvLoRALayer(
       (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     )
     (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (5): ReLU(inplace=True)
     (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     (7): ContinuousConvLoRALayer(
       (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     )
     (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (9): ReLU(inplace=True)
     (10): ContinuousConvLoRALayer(
       (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     )
     (11): Bat

In [52]:
for layer in model.features:
    has_grad = False
    for name, params in layer.named_parameters():
        params.requires_grad = True
        if params.requires_grad:
            has_grad = True

    if has_grad:
        print(layer)
        for name, params in layer.named_parameters():
            print('\t', name, params.requires_grad)
        

for layer in model.classifier:
    has_grad = False
    for name, params in layer.named_parameters():
        params.requires_grad = True
        if params.requires_grad:
            has_grad = True

    if has_grad:
        print(layer)
        for name, params in layer.named_parameters():
            print('\t', name, params.requires_grad)
        

ContinuousConvLoRALayer(
  (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)
	 lora_A True
	 lora_B True
	 conv.weight True
	 conv.bias True
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
	 weight True
	 bias True
ContinuousConvLoRALayer(
  (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)
	 lora_A True
	 lora_B True
	 conv.weight True
	 conv.bias True
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
	 weight True
	 bias True
ContinuousConvLoRALayer(
  (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)
	 lora_A True
	 lora_B True
	 conv.weight True
	 conv.bias True
BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
	 weight True
	 bias True
ContinuousConvLoRALayer(
  (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)
	 lora_A True
	 lora_B True
	 conv.weight True
	 conv.bias True
BatchNorm2d

In [60]:
for name , param in model.named_parameters():
    print(name, param.requires_grad)

features.0.lora_A True
features.0.lora_B True
features.0.conv.weight False
features.0.conv.bias False
features.1.weight True
features.1.bias True
features.3.lora_A True
features.3.lora_B True
features.3.conv.weight False
features.3.conv.bias False
features.4.weight True
features.4.bias True
features.7.lora_A True
features.7.lora_B True
features.7.conv.weight False
features.7.conv.bias False
features.8.weight True
features.8.bias True
features.10.lora_A True
features.10.lora_B True
features.10.conv.weight False
features.10.conv.bias False
features.11.weight True
features.11.bias True
features.14.lora_A True
features.14.lora_B True
features.14.conv.weight False
features.14.conv.bias False
features.15.weight True
features.15.bias True
features.17.lora_A True
features.17.lora_B True
features.17.conv.weight False
features.17.conv.bias False
features.18.weight True
features.18.bias True
features.20.lora_A True
features.20.lora_B True
features.20.conv.weight False
features.20.conv.bias False


In [61]:
torch.cuda.is_available()

True

In [69]:
bz=config['batch_size']
model_type='vgg19'
# device='cpu'
for task in tasks:
    epoch_losses = {'train_acc': [], 'train_loss': [], 'test_acc': [], 'test_loss': []}
    model.to(device)
    print(40*'-', f'TASK_{task}', 40*'-')

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
    lr_scheduler = ReduceLROnPlateau(optimizer, mode='max', patience=4, verbose=True)
    
    early_stop = 0
    best_val = 0
    for i in range(5):
        g = torch.Generator()
        g.manual_seed(0)
        batch_gen = torch.utils.data.DataLoader(train_ds[task], 
                                              batch_size=config['batch_size'], 
                                              shuffle=True, 
                                              num_workers=1,
                                              )
        print(f'Training task {task} in epoch {i}. Batch size: {bz}.')
        count = 0
        total_loss = .0
        total_hit = 0
        for batch in tqdm(batch_gen):
            sample = batch[0].to(device)
            target = batch[1]
            target_onehot = F.one_hot(target, num_classes=total_number_classes).to(torch.float).to(device)

            y_hat = model(sample)

            # Compute the loss
            loss_training = criterion(y_hat, target_onehot)            
            
            # Backpropagation and optimization
            optimizer.zero_grad()
            loss_training.backward()
            optimizer.step()
            
            total_loss += loss_training
            total_hit += sum(np.argmax(y_hat.cpu().detach().numpy(), axis=1) == target.numpy())

            sample.to('cpu')
            target_onehot.to('cpu')
            del sample, target_onehot
            gc.collect()
            torch.cuda.empty_cache()
            
        # Evaluate in test DS after each epoch
        with torch.no_grad():
            batch_gen_test = torch.utils.data.DataLoader(test_ds[task], 
                                              batch_size=config['batch_size'], 
                                              shuffle=True, 
                                              num_workers=1,
                                              )
            model.eval()
            test_loss = 0.
            acc = 0.
            for batch_test in tqdm(batch_gen_test):
                sample = batch_test[0].to(device)
                target = batch_test[1]
                target_onehot = F.one_hot(target, num_classes=total_number_classes).to(torch.float).to(device)
    
                y_hat = model(sample)
                cpu_inference = y_hat.argmax(axis=1)
                
                test_loss += criterion(y_hat, target_onehot)
                acc += sum(cpu_inference.cpu() == target).item()

                sample.to('cpu')
                target_onehot.to('cpu')
                del sample, target_onehot
                gc.collect()
                torch.cuda.empty_cache()

            test_loss = (test_loss/len(batch_gen_test)).cpu()
            acc = acc/len(test_ds[task])
            lr_scheduler.step(acc)
            model.train()
        
        epoch_losses['train_acc'].append(total_hit/(len(batch_gen)*bz))
        epoch_losses['train_loss'].append((total_loss/len(batch_gen)).cpu().item())
        
        epoch_losses['test_acc'].append(acc)
        epoch_losses['test_loss'].append(test_loss.item())

        # if acc > (best_val+0.01):
        if acc > (best_val):
            model.to('cpu')
            best_model[f'{model_type}-{task}'] = deepcopy(model)
            model.to(device)
            best_val = acc
            early_stop = 0

        if early_stop > patience:
            break
        
        early_stop += 1
        
        print(f'Trainig acc: {total_hit/(len(batch_gen)*bz):.4}   //   Training loss: {(total_loss/len(batch_gen)):.4f}   //   Test acc: {acc:.4f}   //   Test loss: {test_loss:.4f}')
        print(f'early_stop: {early_stop}  /   Best acc: {best_val}')
        del batch_gen, batch_gen_test
        gc.collect()
        torch.cuda.empty_cache()

    model.to('cpu')
    model = best_model[f'{model_type}-{task}']
    del best_model[f'{model_type}-{task}']

    gc.collect()
    torch.cuda.empty_cache()
    
    # del criterion, optimizer, lr_scheduler 
    gc.collect()
    torch.cuda.empty_cache()
    target_task = task + 1
    if target_task < len(tasks): 
        model.change_to_task(target_task)


---------------------------------------- TASK_0 ----------------------------------------
Training task 0 in epoch 0. Batch size: 128.


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:15<00:00,  1.46it/s]


Trainig acc: 0.009766   //   Training loss: 5.2958   //   Test acc: 0.0112   //   Test loss: 5.2958
early_stop: 1  /   Best acc: 0.0111731843575419
Training task 0 in epoch 1. Batch size: 128.


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:16<00:00,  1.41it/s]


Trainig acc: 0.01172   //   Training loss: 5.2958   //   Test acc: 0.0115   //   Test loss: 5.2958
early_stop: 1  /   Best acc: 0.011522346368715084
Training task 0 in epoch 2. Batch size: 128.


...

Trainig acc: 0.04818   //   Training loss: 5.2936   //   Test acc: 0.0735   //   Test loss: 5.2936
early_stop: 1  /   Best acc: 0.07346938775510205
Training task 3 in epoch 1. Batch size: 128.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:04<00:00,  1.30it/s]


Trainig acc: 0.0599   //   Training loss: 5.2936   //   Test acc: 0.0639   //   Test loss: 5.2936
early_stop: 2  /   Best acc: 0.07346938775510205
Training task 3 in epoch 2. Batch size: 128.


 50%|███████████████████████████████████████████████████████████████████                                                                   | 3/6 [00:09<00:09,  3.03s/it]


KeyboardInterrupt: 

In [23]:
batch_gen_test = torch.utils.data.DataLoader(test_ds[task], 
                                              batch_size=config['batch_size'], 
                                              shuffle=True, 
                                              num_workers=1,
                                              )
model.eval()
model = model.to(device)
with torch.no_grad():
    for batch_test in tqdm(batch_gen_test):
        sample = batch_test[0].to(device)
        target = batch_test[1]
        target_onehot = F.one_hot(target, num_classes=total_number_classes).to(torch.float).to(device)
    
        y_hat = model(sample)
        cpu_inference = y_hat.argmax(axis=1)
        
        test_loss += criterion(y_hat, target_onehot)
        acc += sum(cpu_inference.cpu() == target).item()
    
        sample.to('cpu')
        target_onehot.to('cpu')
        del sample, target_onehot
        gc.collect()
        torch.cuda.empty_cache()
        break

  0%|                                                                                                                                             | 0/92 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [24]:
from torch import nn, flatten, Tensor

y_hat = model.features(sample)
y_hat = model.avgpool(y_hat)

y_hat = flatten(y_hat, 1)

y_hat = model.classifier(y_hat)

# Mask code
y_hat = model.softmax(y_hat)
y_hat[:, model.masks[model.current_task]] = .0

In [33]:
y_hat[0]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 

In [37]:
model.current_task
len(model.masks[model.current_task])

175

In [69]:
64*3*3*3

1728

In [71]:
192*9 + 9*9

1809

In [68]:
model.features[0].conv.weight.shape

torch.Size([64, 3, 3, 3])

In [10]:
r_lin = 100
lin_params = (r_lin*25088+r_lin*4096)
model.classifier[0].weight.shape, 4096*25088, model.classifier[0].lora_A.shape,  model.classifier[0].lora_B.shape, lin_params, lin_params/(4096*25088)

(torch.Size([4096, 25088]),
 102760448,
 torch.Size([100, 25088]),
 torch.Size([4096, 100]),
 2918400,
 0.0284000318877551)

In [8]:
config = {
    "learning_rate": 0.00001,
    "weight_decay": 5.e-4,
    "batch_size": 8,
    "optimizer": "Adam",
    "architecture": "LoraVGG19",
    "dataset": f'{dataset_name.upper()}',
    "epochs": 10,
    "lr_schedule": "ReduceLROnPlateau - Patience 10 - Monitoring Val Accuracy",
    "description": "Testing CUBs200 splited into 5 tasks."
}

torch.backends.cudnn.benchmark = False
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)
base_model = models.vgg19_bn(weights="IMAGENET1K_V1")
base_model.classifier[6] = nn.Linear(4096, total_number_classes)
model = LoraVGG19(
    model=base_model,
    masks=masks,
    r_conv=9,
    r_linear=30,
    adapt_last_n_conv=0,
    adapt_last_n_linear=3
)
model.classifier[6] = nn.Linear(4096, total_number_classes)

tasks = [i for i in range(n_split_experiences)]
acc_by_task = {i: 0 for i in range(n_split_experiences)}
results_diff_models = {}
best_model = {}
cms = {}
patience = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [47]:
model.classifier[6].requires_grad_()
model.classifier[3].requires_grad_()
model.classifier[0].requires_grad_()
model.classifier[3].bias.requires_grad

True

In [9]:
%%time
# model.features[0].lora_A, model.features[0].lora_B
# model.features[0].a
model

CPU times: user 3 μs, sys: 0 ns, total: 3 μs
Wall time: 5.96 μs


LoraVGG19(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(12

In [10]:
# model.features[0].conv.weight
model.count_features_trainable_params(), model.count_classifier_trainable_params()

# model.classifier[3].count_trainable_parameters()
# model.classifier[6].bias.requires_grad, model.classifier[6].weight.requires_grad

(0, 1940680)

In [50]:
%%time
task = 0
bz=8
model_type='convlora'
epoch_losses = {'train_acc': [], 'train_loss': [], 'test_acc': [], 'test_loss': []}
model.to(device)
print(40*'-', f'TASK_{task}', 40*'-')

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
lr_scheduler = ReduceLROnPlateau(optimizer, mode='max', patience=4, verbose=True)

early_stop = 0
best_val = 0
for i in range(300):
    g = torch.Generator()
    g.manual_seed(0)
    batch_gen = torch.utils.data.DataLoader(train_ds[task], 
                                          batch_size=config['batch_size'], 
                                          shuffle=True, 
                                          num_workers=1,
                                          )
    print(f'Training task {task} in epoch {i}. Batch size: {bz}.')
    count = 0
    total_loss = .0
    total_hit = 0
    for batch in tqdm(batch_gen):
        sample = batch[0].to(device)
        target = batch[1]
        target_onehot = F.one_hot(target, num_classes=total_number_classes).to(torch.float).to(device)

        y_hat = model(sample)

        # Compute the loss
        loss_training = criterion(y_hat, target_onehot)            
        
        # Backpropagation and optimization
        optimizer.zero_grad()
        loss_training.backward()
        optimizer.step()
        
        total_loss += loss_training
        total_hit += sum(np.argmax(y_hat.cpu().detach().numpy(), axis=1) == target.numpy())

        sample.to('cpu')
        target_onehot.to('cpu')
        del sample, target_onehot
        gc.collect()
        torch.cuda.empty_cache()
        
    # Evaluate in test DS after each epoch
    with torch.no_grad():
        batch_gen_test = torch.utils.data.DataLoader(test_ds[task], 
                                          batch_size=config['batch_size'], 
                                          shuffle=True, 
                                          num_workers=1,
                                          )
        model.eval()
        test_loss = 0.
        acc = 0.
        for batch_test in tqdm(batch_gen_test):
            sample = batch_test[0].to(device)
            target = batch_test[1]
            target_onehot = F.one_hot(target, num_classes=total_number_classes).to(torch.float).to(device)

            y_hat = model(sample)
            cpu_inference = y_hat.argmax(axis=1)
            
            test_loss += criterion(y_hat, target_onehot)
            acc += sum(cpu_inference.cpu() == target).item()

            sample.to('cpu')
            target_onehot.to('cpu')
            del sample, target_onehot
            gc.collect()
            torch.cuda.empty_cache()

        test_loss = (test_loss/len(batch_gen_test)).cpu()
        acc = acc/len(test_ds[task])
        lr_scheduler.step(acc)
        model.train()
    
    epoch_losses['train_acc'].append(total_hit/(len(batch_gen)*bz))
    epoch_losses['train_loss'].append((total_loss/len(batch_gen)).cpu().item())
    
    epoch_losses['test_acc'].append(acc)
    epoch_losses['test_loss'].append(test_loss.item())

    # if acc > (best_val+0.01):
    if acc > (best_val):
        # model.to('cpu')
        # best_model[f'{model_type}-{task}'] = deepcopy(model)
        # model.to(device)
        best_val = acc
        early_stop = 0

    if early_stop > patience:
        break
    
    early_stop += 1
    
    print(f'Trainig acc: {total_hit/(len(batch_gen)*bz):.4}   //   Training loss: {(total_loss/len(batch_gen)):.4f}   //   Test acc: {acc:.4f}   //   Test loss: {test_loss:.4f}')
    print(f'early_stop: {early_stop}  /   Best acc: {best_val}')
    del batch_gen, batch_gen_test
    gc.collect()
    torch.cuda.empty_cache()

# model.to('cpu')
# model = best_model[f'{model_type}-{task}']
# del best_model[f'{model_type}-{task}']

gc.collect()
torch.cuda.empty_cache()

# del criterion, optimizer, lr_scheduler 
gc.collect()
torch.cuda.empty_cache()
target_task = task + 1

---------------------------------------- TASK_0 ----------------------------------------
Training task 0 in epoch 0. Batch size: 8.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 358/358 [01:21<00:00,  4.42it/s]


Trainig acc: 0.04633   //   Training loss: 5.2792   //   Test acc: 0.1561   //   Test loss: 5.2033
early_stop: 1  /   Best acc: 0.1560754189944134
Training task 0 in epoch 1. Batch size: 8.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 358/358 [01:20<00:00,  4.42it/s]


Trainig acc: 0.1627   //   Training loss: 5.1851   //   Test acc: 0.2559   //   Test loss: 5.1029
early_stop: 1  /   Best acc: 0.25593575418994413
Training task 0 in epoch 2. Batch size: 8.


...

Trainig acc: 0.7603   //   Training loss: 4.5626   //   Test acc: 0.6054   //   Test loss: 4.7265
early_stop: 10  /   Best acc: 0.6166201117318436
Training task 0 in epoch 72. Batch size: 8.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 358/358 [01:17<00:00,  4.64it/s]


Trainig acc: 0.7587   //   Training loss: 4.5638   //   Test acc: 0.6023   //   Test loss: 4.7340
early_stop: 11  /   Best acc: 0.6166201117318436
Training task 0 in epoch 73. Batch size: 8.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 358/358 [01:16<00:00,  4.69it/s]


CPU times: user 3h 34min 22s, sys: 4min 16s, total: 3h 38min 38s
Wall time: 3h 39min 2s


In [14]:
model

LoraVGG19(
  (features): Sequential(
    (0): Conv2dLora(3, 64, kernel_size=3, stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2dLora(64, 64, kernel_size=3, stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2dLora(64, 128, kernel_size=3, stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2dLora(128, 128, kernel_size=3, stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2dLora(12

In [77]:
model.features[0].lora_A, model.features[0].lora_B

(Parameter containing:
 tensor([[-0.0378,  0.0567,  0.1117, -0.0339,  0.0553, -0.0040, -0.0299,  0.0211,
          -0.0835],
         [-0.0905,  0.1162,  0.0151, -0.0443, -0.0483,  0.1258, -0.0734,  0.1352,
          -0.1183],
         [-0.1429,  0.0847,  0.0833,  0.0792, -0.0334, -0.0474,  0.0179,  0.0147,
          -0.0258],
         [-0.0596, -0.1432,  0.0600,  0.0647,  0.1609, -0.1108, -0.0214,  0.1754,
          -0.1787],
         [ 0.1239, -0.0960, -0.0290,  0.0268, -0.0605,  0.1394, -0.0755, -0.0516,
           0.0481],
         [-0.0926, -0.0775, -0.0588, -0.0384,  0.1045,  0.0654,  0.0716, -0.0614,
           0.0230],
         [-0.0505,  0.0856,  0.1879, -0.0491, -0.2247,  0.0297,  0.1345, -0.0377,
          -0.0010],
         [-0.0702,  0.0286, -0.1730,  0.0926,  0.0122,  0.0460, -0.1009,  0.1071,
          -0.0044],
         [ 0.0469, -0.0567, -0.1212, -0.0473, -0.2061,  0.1177,  0.1110,  0.0067,
           0.0723]], requires_grad=True),
 Parameter containing:
 tensor([[ 2.1

In [78]:
model.features[0].conv.weight

Parameter containing:
tensor([[[[-3.2130e-08, -3.1717e-08, -6.7120e-08],
          [-2.9321e-08, -2.1596e-08, -5.3207e-08],
          [-5.3304e-08, -4.0832e-08, -6.6434e-08]],

         [[ 1.3710e-07,  1.2848e-07,  7.1270e-08],
          [ 1.5487e-07,  1.5186e-07,  8.3582e-08],
          [ 9.3392e-08,  9.9451e-08,  4.1284e-08]],

         [[ 1.7435e-07,  1.7840e-07,  1.0561e-07],
          [ 1.9644e-07,  1.9769e-07,  1.2560e-07],
          [ 1.1935e-07,  1.3447e-07,  5.9638e-08]]],


        [[[-6.3467e-08, -5.2714e-08, -3.8511e-08],
          [-1.4349e-08, -9.0426e-09, -1.7164e-08],
          [ 2.2282e-08,  1.9084e-08,  1.8150e-09]],

         [[-1.2191e-08, -6.1933e-09,  8.5022e-09],
          [ 3.8618e-08,  5.8869e-08,  2.2082e-08],
          [ 6.2501e-08,  7.0528e-08,  2.7185e-08]],

         [[ 3.0045e-08,  3.8003e-08,  3.5389e-08],
          [ 8.6743e-08,  1.1626e-07,  8.0502e-08],
          [ 7.9653e-08,  1.0057e-07,  6.7894e-08]]],


        [[[-1.3565e-01, -2.9960e-01, -1.5151

In [11]:
3

Trainig acc: 7.438   //   Training loss: 4.8364   //   Test acc: 0.4036   //   Test loss: 4.9151
early_stop: 11  /   Best acc: 0.4057262569832402

SyntaxError: invalid syntax (1096729732.py, line 3)

In [12]:
30

Trainig acc: 9.818   //   Training loss: 4.6811   //   Test acc: 0.5112   //   Test loss: 4.8094
early_stop: 11  /   Best acc: 0.5115223463687151
Training task 0 in epoch 81. Batch size: 8.


SyntaxError: invalid syntax (637401105.py, line 3)

In [11]:
old -> Treinava todas as Conv Layers , nao usava a Lora

Trainig acc: 0.973   //   Training loss: 4.3385   //   Test acc: 0.7678   //   Test loss: 4.5556
early_stop: 11  /   Best acc: 0.7730446927374302


SyntaxError: invalid syntax (3970758156.py, line 3)

In [12]:
ContinuousConvLora - R=3

Trainig acc: 0.9033   //   Training loss: 4.4146   //   Test acc: 0.7360   //   Test loss: 4.5894
early_stop: 11  /   Best acc: 0.7409217877094972
Training task 0 in epoch 92. Batch size: 8.


SyntaxError: invalid syntax (3907677468.py, line 3)

In [ ]:
old (changed) - Congelei as conv e treinei apenas as lora

Trainig acc: 0.9207   //   Training loss: 4.3934   //   Test acc: 0.7497   //   Test loss: 4.5735
early_stop: 11  /   Best acc: 0.7548882681564246
Training task 0 in epoch 83. Batch size: 8.

In [ ]:
ContinuousConvLora - R=1

Trainig acc: 0.784   //   Training loss: 4.5358   //   Test acc: 0.6568   //   Test loss: 4.6754
early_stop: 11  /   Best acc: 0.6588687150837989

In [3]:
from torch import nn
from continuous_lora.layers.continuous_conv_lora_layer import ContinuousConvLoRALayer
from continuous_lora.layers.old_conv_lora_layer import Conv2dLora



in_channels=[3,64,64,128,128,256,256,512]
out_channels=[64,64,128,128,256,256,512,512]

for in_channel,out_channel in zip(in_channels,out_channels):
    print(3*'\n'+f'in_channel: {in_channel} | out_channel {out_channel}')
    kernel_size=3
    
    r=25
    
    
    normal_conv = nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=kernel_size)
    normal_conv_num_params = 0
    for name, p in normal_conv.named_parameters(): 
        if p.requires_grad:
            normal_conv_num_params += p.numel()
            print(f'{name} é um parametro treinável. {p.numel()}. {p.shape}')
        else:
            print(f'{name} NÃO é um parametro treinável')
    
    print(f'Numero de parametros treinaveis na layer Conv normal: {normal_conv_num_params}')
    
    print(3*'\n'+40*'-'+'Continuous Conv Lora')
    continuous_conv = ContinuousConvLoRALayer(in_channels=in_channel, out_channels=out_channel, kernel_size=kernel_size, number_of_tasks=1, conv_module=nn.Conv2d, r=r)
    continuous_conv_num_params = 0
    for name, p in continuous_conv.named_parameters(): 
        if p.requires_grad:
            continuous_conv_num_params += p.numel()
            print(f'{name} é um parametro treinável. {p.numel()}. {p.shape}')
        else:
            print(f'{name} NÃO é um parametro treinável')
    
    print(f'Numero de parametros treinaveis ConvLora terceiro: {continuous_conv_num_params}')
    
    print(3*'\n'+40*'-'+'Continuous Conv Lora NOSSO')
    continuous_conv_our = Conv2dLora(in_channels=in_channel, out_channels=out_channel, kernel_size=kernel_size)
    continuous_conv_our.weight.requires_grad = False
    continuous_conv_our.bias.requires_grad = False
    continuous_conv_our_num_params = 0
    for name, p in continuous_conv_our.named_parameters(): 
        if p.requires_grad:
            continuous_conv_our_num_params += p.numel()
            print(f'{name} é um parametro treinável. {p.numel()}. {p.shape}')
        else:
            print(f'{name} NÃO é um parametro treinável')
    
    print(f'Numero de parametros treinaveis no nosso ConvLora: {continuous_conv_our_num_params}')





in_channel: 3 | out_channel 64
weight é um parametro treinável. 1728. torch.Size([64, 3, 3, 3])
bias é um parametro treinável. 64. torch.Size([64])
Numero de parametros treinaveis na layer Conv normal: 1792



----------------------------------------Continuous Conv Lora
lora_A é um parametro treinável. 675. torch.Size([75, 9])
lora_B é um parametro treinável. 14400. torch.Size([192, 75])
conv.weight NÃO é um parametro treinável
conv.bias é um parametro treinável. 64. torch.Size([64])
Numero de parametros treinaveis ConvLora terceiro: 15139



----------------------------------------Continuous Conv Lora NOSSO
weight NÃO é um parametro treinável
bias NÃO é um parametro treinável
a é um parametro treinável. 9. torch.Size([1, 3, 3, 1])
b é um parametro treinável. 9. torch.Size([1, 3, 1, 3])
c é um parametro treinável. 192. torch.Size([64, 3, 1, 1])
d é um parametro treinável. 64. torch.Size([64])
d_ é um parametro treinável. 1. torch.Size([1])
Numero de parametros treinaveis no nosso Co

In [3]:
import sys
import os

# Adiciona o diretório pai ao caminho de pesquisa de módulos
sys.path.append(os.path.abspath(os.path.join('..')))

from torch import nn
from continuous_lora.models.lora_vgg19 import LoraVGG19
import torchvision.models as models

base_model = models.vgg19_bn(weights="IMAGENET1K_V1")
base_model.classifier[6] = nn.Linear(4096, 200)
model = LoraVGG19(
    model=base_model,
    masks=[[i for i in range(200)]],
    r_conv=1,
    r_linear=100,
    adapt_last_n_conv=0,
    adapt_last_n_linear=0
)
print('APENAS CONTANDO OS PARAMETROS DAS LAYERS CONVOLUCIONAIS. PRECISEI ALTERAR O CODIGO PARA NÃO CONGELAR AS LAYERS\n\n')
print(f'Número de parametros treinaveis no features extractor: {model.count_features_trainable_params()}')
print(f'Número de parametros treinaveis no classifier: {model.count_classifier_trainable_params()}')
print(f'Número de parametros treinaveis total: {model.count_trainable_params()}')

APENAS CONTANDO OS PARAMETROS DAS LAYERS CONVOLUCIONAIS. PRECISEI ALTERAR O CODIGO PARA NÃO CONGELAR AS LAYERS


Número de parametros treinaveis no features extractor: 20024384
Número de parametros treinaveis no classifier: 0
Número de parametros treinaveis total: 20024384


In [2]:
import sys
import os

# Adiciona o diretório pai ao caminho de pesquisa de módulos
sys.path.append(os.path.abspath(os.path.join('..')))

from torch import nn
from continuous_lora.models.lora_vgg19 import LoraVGG19
import torchvision.models as models

base_model = models.vgg19_bn(weights="IMAGENET1K_V1")
base_model.classifier[6] = nn.Linear(4096, 200)
model = LoraVGG19(
    model=base_model,
    masks=[[i for i in range(200)]],
    r_conv=1,
    r_linear=100,
    adapt_last_n_conv=16,
    adapt_last_n_linear=0
)

print('APENAS CONTANDO OS PARAMETROS DOS FILTROS A,B,C e D (nosso). PRECISEI ALTERAR O CODIGO PARA USAR NOSSA IMPL.\n\n')
print(f'Número de parametros treinaveis no features extractor: {model.count_features_trainable_params()}')
print(f'Número de parametros treinaveis no classifier: {model.count_classifier_trainable_params()}')
print(f'Número de parametros treinaveis total: {model.count_trainable_params()}')

APENAS CONTANDO OS PARAMETROS DOS FILTROS A,B,C e D (nosso). PRECISEI ALTERAR O CODIGO PARA USAR NOSSA IMPL.


Número de parametros treinaveis no features extractor: 2259810
Número de parametros treinaveis no classifier: 0
Número de parametros treinaveis total: 2259810


In [1]:
import sys
import os

# Adiciona o diretório pai ao caminho de pesquisa de módulos
sys.path.append(os.path.abspath(os.path.join('..')))

from torch import nn
from continuous_lora.models.lora_vgg19 import LoraVGG19
import torchvision.models as models

base_model = models.vgg19_bn(weights="IMAGENET1K_V1")
base_model.classifier[6] = nn.Linear(4096, 200)
model = LoraVGG19(
    model=base_model,
    masks=[[i for i in range(200)]],
    r_conv=1,
    r_linear=100,
    adapt_last_n_conv=16,
    adapt_last_n_linear=0
)

print('APENAS CONTANDO OS PARAMETROS DOS FILTROS A,B (NOVA IMPLEMENTACAO) R=1. PRECISEI ALTERAR O CODIGO PARA USAR NOSSA IMPL.\n\n')
print(f'Número de parametros treinaveis no features extractor: {model.count_features_trainable_params()}')
print(f'Número de parametros treinaveis no classifier: {model.count_classifier_trainable_params()}')
print(f'Número de parametros treinaveis total: {model.count_trainable_params()}')

APENAS CONTANDO OS PARAMETROS DOS FILTROS A,B (NOVA IMPLEMENTACAO) R=1. PRECISEI ALTERAR O CODIGO PARA USAR NOSSA IMPL.


Número de parametros treinaveis no features extractor: 94491
Número de parametros treinaveis no classifier: 0
Número de parametros treinaveis total: 94491


In [2]:
import sys
import os

# Adiciona o diretório pai ao caminho de pesquisa de módulos
sys.path.append(os.path.abspath(os.path.join('..')))

from torch import nn
from continuous_lora.models.lora_vgg19 import LoraVGG19
import torchvision.models as models

base_model = models.vgg19_bn(weights="IMAGENET1K_V1")
base_model.classifier[6] = nn.Linear(4096, 200)
model = LoraVGG19(
    model=base_model,
    masks=[[i for i in range(200)]],
    r_conv=3,
    r_linear=100,
    adapt_last_n_conv=16,
    adapt_last_n_linear=0
)

print('APENAS CONTANDO OS PARAMETROS DOS FILTROS A,B (NOVA IMPLEMENTACAO) R=3. PRECISEI ALTERAR O CODIGO PARA USAR NOSSA IMPL.\n\n')
print(f'Número de parametros treinaveis no features extractor: {model.count_features_trainable_params()}')
print(f'Número de parametros treinaveis no classifier: {model.count_classifier_trainable_params()}')
print(f'Número de parametros treinaveis total: {model.count_trainable_params()}')

APENAS CONTANDO OS PARAMETROS DOS FILTROS A,B (NOVA IMPLEMENTACAO) R=3. PRECISEI ALTERAR O CODIGO PARA USAR NOSSA IMPL.


Número de parametros treinaveis no features extractor: 283473
Número de parametros treinaveis no classifier: 0
Número de parametros treinaveis total: 283473


In [5]:
import sys
import os

# Adiciona o diretório pai ao caminho de pesquisa de módulos
sys.path.append(os.path.abspath(os.path.join('..')))

from torch import nn
from continuous_lora.models.lora_vgg19 import LoraVGG19
import torchvision.models as models

base_model = models.vgg19_bn(weights="IMAGENET1K_V1")
base_model.classifier[6] = nn.Linear(4096, 200)
model = LoraVGG19(
    model=base_model,
    masks=[[i for i in range(200)]],
    r_conv=24,
    r_linear=100,
    adapt_last_n_conv=16,
    adapt_last_n_linear=0
)

print('APENAS CONTANDO OS PARAMETROS DOS FILTROS A,B (NOVA IMPLEMENTACAO) R=24. PRECISEI ALTERAR O CODIGO PARA USAR NOSSA IMPL.\n\n')
print(f'Número de parametros treinaveis no features extractor: {model.count_features_trainable_params()}')
print(f'Número de parametros treinaveis no classifier: {model.count_classifier_trainable_params()}')
print(f'Número de parametros treinaveis total: {model.count_trainable_params()}')

APENAS CONTANDO OS PARAMETROS DOS FILTROS A,B (NOVA IMPLEMENTACAO) R=24. PRECISEI ALTERAR O CODIGO PARA USAR NOSSA IMPL.


Número de parametros treinaveis no features extractor: 2267784
Número de parametros treinaveis no classifier: 0
Número de parametros treinaveis total: 2267784


In [20]:
import sys
import os

# Adiciona o diretório pai ao caminho de pesquisa de módulos
sys.path.append(os.path.abspath(os.path.join('..')))

from torch import nn
from continuous_lora.models.lora_vgg19 import LoraVGG19
import torchvision.models as models

base_model = models.vgg19_bn(weights="IMAGENET1K_V1")
base_model.classifier[6] = nn.Linear(4096, 200)
model = LoraVGG19(
    model=base_model,
    masks=[[i for i in range(200)]],
    r_conv=12,
    r_linear=100,
    adapt_last_n_conv=16,
    adapt_last_n_linear=0
)

print('APENAS CONTANDO OS PARAMETROS DOS FILTROS A,B (NOVA IMPLEMENTACAO) R=12. PRECISEI ALTERAR O CODIGO PARA USAR NOSSA IMPL.\n\n')
print(f'Número de parametros treinaveis no features extractor: {model.count_features_trainable_params()}')
print(f'Número de parametros treinaveis no classifier: {model.count_classifier_trainable_params()}')
print(f'Número de parametros treinaveis total: {model.count_trainable_params()}')

APENAS CONTANDO OS PARAMETROS DOS FILTROS A,B (NOVA IMPLEMENTACAO) R=12. PRECISEI ALTERAR O CODIGO PARA USAR NOSSA IMPL.


Número de parametros treinaveis no features extractor: 1133892
Número de parametros treinaveis no classifier: 0
Número de parametros treinaveis total: 1133892
